In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

from gtda.homology import CubicalPersistence
from gtda.diagrams import PersistenceImage, PersistenceLandscape

X_img = np.load("/home/sajedhamdan/Desktop/skin_cancer/images_train_256x192.npy")
y = np.load("/home/sajedhamdan/Desktop/skin_cancer/train_labels.npy")
X_img = X_img.astype(np.float32)

sample_size = 1000
X_img, _, y, _ = train_test_split(X_img, y, train_size=sample_size, stratify=y, random_state=42)

# TDA Feature Extraction
def extract_tda_features(X_rgb):
    X_gray = 0.2989 * X_rgb[..., 0] + 0.5870 * X_rgb[..., 1] + 0.1140 * X_rgb[..., 2]
    cp = CubicalPersistence(homology_dimensions=[0, 1], n_jobs=-1)
    diagrams = cp.fit_transform(X_gray)

    pi = PersistenceImage(sigma=1.0, n_bins=20, weight_function=lambda x: x[1] ** 2)
    pi_feat = pi.fit_transform(diagrams).reshape(len(diagrams), -1)

    pl = PersistenceLandscape(n_layers=5, n_bins=50)
    pl_feat = pl.fit_transform(diagrams).reshape(len(diagrams), -1)

    return np.hstack((pi_feat, pl_feat))

print("Extracting TDA features...")
X_tda = extract_tda_features(X_img)
print("TDA shape:", X_tda.shape)

smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X_tda, y)

# Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_bal)
y_cat = to_categorical(y_bal)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_bal), y=y_bal)
class_weight_dict = dict(enumerate(class_weights))

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)

# MLP Model (ResNet-style)
input_layer = Input(shape=(X_scaled.shape[1],))

# block 1
x = Dense(512, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
res1 = Dense(512)(x)
res1 = BatchNormalization()(res1)

# block 2
x = Dense(512, activation='relu')(res1)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
res2 = Add()([x, res1]) 

# block 3
x = Dense(256, activation='relu')(res2)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
res3 = Dense(256)(x)
res3 = BatchNormalization()(res3)
x = Add()([x, res3])  

# output layer
output_layer = Dense(7, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stop, lr_schedule],
    verbose=1
)

results = model.evaluate(X_test, y_test, verbose=1)
print(f"\nTest - Accuracy: {results[1]:.4f} | Precision: {results[2]:.4f} | Recall: {results[3]:.4f}")

model.save("tda_resnet_model_v2.keras")


2025-06-08 13:57:38.661504: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-08 13:57:38.822126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749380258.880816   51010 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749380258.899431   51010 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749380259.032113   51010 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Extracting TDA features...
TDA shape: (1000, 1300)
Epoch 1/100


2025-06-08 13:58:25.611157: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.3146 - loss: 2.4115 - precision: 0.3643 - recall: 0.2530 - val_accuracy: 0.5550 - val_loss: 1.2684 - val_precision: 0.6753 - val_recall: 0.4173 - learning_rate: 1.0000e-04
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5371 - loss: 1.4416 - precision: 0.5956 - recall: 0.4566 - val_accuracy: 0.6403 - val_loss: 1.0952 - val_precision: 0.7175 - val_recall: 0.5123 - learning_rate: 1.0000e-04
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5817 - loss: 1.2702 - precision: 0.6470 - recall: 0.5096 - val_accuracy: 0.7332 - val_loss: 0.8499 - val_precision: 0.8050 - val_recall: 0.6211 - learning_rate: 1.0000e-04
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6262 - loss: 1.0939 - precision: 0.6786 - recall: 0.5603 - val_accuracy: 0.7481 - val_loss: 0.7856 - val_precision: 0.8042 - val_recall: 0.6617 - learning_rate: 1.0000e-04
Epoch 5/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accura